In [84]:
import os, glob, time, pandas as pd

In [85]:
def get_file_last_time_of_change(fname):
    try:
        mtime = os.path.getmtime(fname)
    except OSError:
        mtime = 0
    return mtime

In [86]:
def process_changed_files(list_of_changed_files):
    for fname in list_of_changed_files:
        if '0~' in fname:
            update_trade(fname)
        elif '2~' in fname:
            update_current(fname)

In [87]:
################# UPDATING ##################

In [88]:
main_batch_max_len = 300   # 5 min
main_batch = pd.DataFrame({'Timestamp': 0, 'curr_BTC_USD': 0, 'curr_ETH_USD': 0,
                           'curr_LTC_USD': 0, 'curr_ETH_BTC': 0, 'curr_LTC_BTC': 0,
                           'tr_BTC_USD_buy': 0, 'tr_ETH_USD_buy': 0, 'tr_LTC_USD_buy': 0,
                           'tr_ETH_BTC_buy': 0, 'tr_LTC_BTC_buy': 0, 'tr_BTC_USD_sell': 0,
                           'tr_ETH_USD_sell': 0, 'tr_LTC_USD_sell': 0, 'tr_ETH_BTC_sell': 0,
                           'tr_LTC_BTC_sell': 0}, index = [0])
fname_dict = {'2~Poloniex~BTC~USD.dat': 'curr_BTC_USD', '2~Poloniex~ETH~BTC.dat': 'curr_ETH_BTC',
              '2~Poloniex~ETH~USD.dat': 'curr_ETH_USD', '2~Poloniex~LTC~BTC.dat': 'curr_LTC_BTC',
              '2~Poloniex~LTC~USD.dat': 'curr_LTC_USD'}

In [89]:
def load_trade(fname):
    return pd.read_csv(fname, sep=";", header = None, names = ['Timestamp', 'Flag', 'Total'])

In [90]:
def load_current(fname):
    return pd.read_csv(fname, sep=";", header = None, names = ['Timestamp', 'Value'])

In [91]:
def update_trade(fname):
    print(time.asctime(time.localtime(time.time())), 'Updating trade:', fname)
    df_new_trade = time_repair_trade(load_trade(fname))

In [92]:
def update_current(fname):
    print(time.asctime(time.localtime(time.time())), 'Updating current:', fname)
    df_new_current = time_repair_current(load_current(fname))
    print(df_new_current)

In [93]:
def get_t_list(df):
    return list(range(int(min(df.Timestamp)), int(max(df.Timestamp))))

In [98]:
def time_repair_trade(df_new_trade):
    t_list = get_t_list(df_new_trade)
    #df_new_trade_repaired = pd.DataFrame({''})

In [117]:
def time_repair_current(df_new_current):
    t_list = get_t_list(df_new_current)
    df_new_current_repaired = pd.DataFrame({'Timestamp': t_list, 'Value': 0})
    #df_new_current_repaired[df_new_current_repaired.Timestamp == df_new_current.Timestamp].Value = df_new_current.Value
    return pd.merge(df_new_current, df_new_current_repaired, on=['Timestamp'])
    #for tt in t_list:
    #    df_new_current_repaired.Price[df_current_repaired.Timestamp == tt] = df_new_current[df_new_current.Timestamp == tt].Price

In [118]:
################# MAIN LOOP ##################

In [119]:
last_changed_times = []
last_changed_times_tmp = []
list_of_changed_files = []
last_time = 0

time_divider = 60 #60*10 ... ten minutes

last_time = -1

while True: #for i in range(120):     #while True:
    now_time = str(round(time.time()/time_divider))
    time_changed = False
    if last_time != now_time:
        last_changed_times_tmp = []
        time_changed = True
        last_time = now_time
        files_name = glob.glob('.\\data\\' + last_time + '\\*.dat')
        for fname in files_name:
            last_change_time_file = get_file_last_time_of_change(fname)
            last_changed_times_tmp.append(last_change_time_file)
        
    if not time_changed:
        files_name = glob.glob('.\\data\\' + now_time + '\\*.dat')

    for fname in files_name:
        last_changed_times.append(get_file_last_time_of_change(fname))

    if len(last_changed_times) > len(last_changed_times_tmp):
        last_changed_times_tmp = last_changed_times
        
    for j in range(len(files_name)):
        if last_changed_times_tmp[j] != last_changed_times[j]:
            list_of_changed_files.append(files_name[j])
    last_changed_times_tmp = last_changed_times
    last_changed_times = []
    
    process_changed_files(list_of_changed_files)
    list_of_changed_files = []
    
    time.sleep(1)   

Wed Jun 28 13:17:43 2017 Updating trade: .\data\24977478\0~Poloniex~LTC~USD.dat
Wed Jun 28 13:17:43 2017 Updating current: .\data\24977478\2~Poloniex~LTC~USD.dat
    Timestamp   Value_x  Value_y
0  1498648607  39.11191        0
1  1498648611  39.11191        0
Wed Jun 28 13:17:45 2017 Updating trade: .\data\24977478\0~Poloniex~LTC~BTC.dat
Wed Jun 28 13:17:45 2017 Updating current: .\data\24977478\2~Poloniex~LTC~BTC.dat
     Timestamp   Value_x  Value_y
0   1498648624  0.015924        0
1   1498648627  0.015924        0
2   1498648629  0.015924        0
3   1498648635  0.015924        0
4   1498648636  0.015924        0
5   1498648641  0.015925        0
6   1498648641  0.015929        0
7   1498648643  0.015929        0
8   1498648643  0.015935        0
9   1498648643  0.015938        0
10  1498648643  0.015938        0
11  1498648643  0.015940        0
12  1498648643  0.015941        0
13  1498648643  0.015945        0
14  1498648643  0.015948        0
15  1498648643  0.015950        0

KeyboardInterrupt: 

In [ ]:
translate_fname = {'2~Poloniex~BTC~USD.dat': 'curr_BTC_USD', '2~Poloniex~ETH~BTC.dat': 'curr_ETH_BTC',
                   '2~Poloniex~ETH~USD.dat': 'curr_ETH_USD', '2~Poloniex~LTC~BTC.dat': 'curr_LTC_BTC',
                   '2~Poloniex~LTC~USD.dat': 'curr_LTC_USD'}

In [ ]:
translate_fname['2~Poloniex~BTC~USD.dat']

In [ ]:
asasdasdasdasd